<a href="https://colab.research.google.com/github/Yashvardhan007/gooapp/blob/master/Classwell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-requisites

##### Install libraries

In [ ]:
%%capture
!pip install welly
!pip install catboost

# requirements.txt karna hai

##### Import

In [ ]:
import numpy as np 
import pandas as pd 

import os

import welly
from welly import Well
from welly import Project

import lasio 
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import xgboost as xg
from catboost import CatBoostRegressor
import random

from sklearn.metrics import mean_squared_error as MSE

import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed = 42
seed_everything(seed)

##### Alias Dictionary

In [ ]:
# False dict
alias_dict_F = dict()

alias_dict_F['li_DEC_D'] = ['DPHZ', 'DPHI_LS', 'DPHI', 'DPOR_LS', 'DPO_LS', 'DPHZ_LS'] # from density
alias_dict_F['li_DEC_N'] = ['NPOR_LS', 'ENPH_LS', 'NPHI_LS', 'APLC_LS', 'NPHI_LS_1', 'NPHI', 'SNP'] # from Neutron
alias_dict_F['li_DEC_T'] = ['TNPH_LS_1', 'TNPH_LS', 'TPHI_LS', 'TNPH', 'CNPOR_LS', 'TNPH_LIM'] # thermal
alias_dict_F['li_DEC_S'] = ['SPHI', 'SPHI_LS'] # sonic
alias_dict_F['li_DEC_NOR'] = ['PORS_LS', 'PXND', 'PORZ_LS'] # normal

alias_dict_F['li_USF_S'] = ['DTS', 'DT4ST', 'DT4SR', 'DT1', 'DT4S', 'DTSH', 'DT1T', 'DTTS', 'DTRS', 'DT1R', 'DT2R', 'DTST', 'DT2'] # shear
alias_dict_F['li_USF_P'] = ['DTCO','DT4PR', 'DT4PT', 'DTCO_1', 'DT4P', 'DTTP'] # compressional
alias_dict_F['li_USF_LO'] = ['DTC', 'DT', 'DTLN', 'DTLF', 'DTL'] # long spacing 

alias_dict_F['li_GAPI_D'] = ['GRD1', 'CGRD', 'GRD', 'HCGRD', 'HSGRD'] # density
alias_dict_F['li_GAPI_R'] = ['HCGRR', 'CGRR', 'GRR', 'GRR_R', 'GRR_R2', 'GRR_R1'] # Resistivity
alias_dict_F['li_GAPI_S'] = ['GRS1', 'CGRS', 'ECGRS', 'HSGRS', 'HCGRS'] # Sonic
alias_dict_F['li_GAPI_N'] = ['HGRT', 'GRC', 'GR_EDTC', 'GR', 'ECGREDTC', 'GRT'] # Normal
alias_dict_F['li_GAPI_SPEC'] = ['SGRR', 'HSGR', 'SGRD', 'SGRDD', 'SGR', 'SGRS'] # spectroscopy

alias_dict_F['li_IN_D'] = ['LCALD', 'CALD', 'CALI', 'LCAL', 'LCAL_1'] # density
alias_dict_F['li_IN_R'] = ['CALSR', 'CAL1R', 'CAL1R_R', 'CALSR_R', 'CALR'] # resistivity
alias_dict_F['li_IN_HILT'] = ['HCALS', 'HCAL', 'HCAL_2', 'HCALD', 'HCALR', 'HCAL_1'] # hilt
alias_dict_F['li_IN_HD'] = ['HD2', 'HD1', 'HD', 'HD1_PPC1', 'HDS', 'HD2_PPC2', 'HD2_PPC1'] # hd
alias_dict_F['li_IN_N'] = ['CALX', 'C1', 'C2', 'DCAL'] # normal

alias_dict_F['li_SH'] = ['AE10','AF10','AHT10','AT10','HLLS','HMIN','HMNO','LLS','LLS_R','LLS_R1','LLS_R2','MSFL','MSFL_R','RXO8','SFLU']
alias_dict_F['li_DP'] =['AHT90', 'AF90', 'ILD', 'AT90', 'HRID', 'AE90', 'LLD_R1', 'LLD_R2', 'RT_HRLT', 'LLD', 'HLLD', 'LLD_R']
alias_dict_F['li_MD'] = ['AE20','AE30','AF20','AHT20','AHT30','AT20','AT30','AO30','ILM','TBIT20','ILM','ILM_1','IMBC','IMPH','RXOZ','RXOZ_R','RXO_HRLT']
alias_dict_F['li_RLA'] = ['RLA1','RLA2','RLA3','RLA4','RLA5']

alias_dict_F['li_BE'] = ['PEFL', 'PEFZ', 'PE', 'PEF']

alias_dict_F['li_MV'] = ['SPR']

alias_dict_F['li_LBF_D'] = ['TEND_1', 'TEND1', 'TEND'] # density
alias_dict_F['li_LBF_R'] = ['TENR_R', 'TENR_R', 'TENR_R2', 'TENR', 'TENR_R1'] # resistivity
alias_dict_F['li_LBF_S'] = ['TENS1', 'LTEN', 'TENS'] # sonic
alias_dict_F['li_LBF_N'] = ['TENT', 'TEN'] # normal


alias_F = {"DTSM" :['DTSM'],
         "DEC_D"  : alias_dict_F['li_DEC_D'],
         "DEC_N"  : alias_dict_F['li_DEC_N'],
         "DEC_T"  : alias_dict_F['li_DEC_T'],
         "DEC_S"  : alias_dict_F['li_DEC_S'],
         "DEC_NOR"  : alias_dict_F['li_DEC_NOR'],

         "USF_S"  : alias_dict_F['li_USF_S'],
         "USF_P"  : alias_dict_F['li_USF_P'],
         "USF_LO"  : alias_dict_F['li_USF_LO'],

         "GAPI_D" : alias_dict_F['li_GAPI_D'],
         "GAPI_R" : alias_dict_F['li_GAPI_R'],
         "GAPI_S" : alias_dict_F['li_GAPI_S'],
         "GAPI_N" : alias_dict_F['li_GAPI_N'],
         "GAPI_SPEC" : alias_dict_F['li_GAPI_SPEC'],

         "IN_D"   : alias_dict_F['li_IN_D'],
         "IN_R"   : alias_dict_F['li_IN_R'],
         "IN_HILT"   : alias_dict_F['li_IN_HILT'],
         "IN_N"   : alias_dict_F['li_IN_N'],

         "MV"   : alias_dict_F['li_MV'],

         "BE"   : alias_dict_F['li_BE'],

         "LBF_D"  : alias_dict_F['li_LBF_D'],
         "LBF_R"  : alias_dict_F['li_LBF_R'],
         "LBF_S"  : alias_dict_F['li_LBF_S'],
         "LBF_N"  : alias_dict_F['li_LBF_N'],

         "SH"   : alias_dict_F['li_SH'],
         "DP"   : alias_dict_F['li_DP'], 
         "RL"   : alias_dict_F['li_RLA']
 }
#########################################################

# True dict 
alias_dict_T = dict()
alias_dict_T['li_SH'] = ['AT10', 'AE10', 'AHT10', 'LLS', 'SFL', 'AST10'] # shallow
alias_dict_T['li_MD'] = ['AE20', 'IMBC', 'AF20','AE30', 'ILM_1', 'AST30', 'IMPH', 'AHT30','ILM', 'AHT20', 'TBIT20', 'TBIT30'] # Medium
alias_dict_T['li_DP'] = ['ILD1', 'ILD_1', 'AT90', 'AE90', 'AST90', 'LLD', 'ILD', 'AHT90', 'IDPH'] # deep
alias_dict_T['li_SFL'] = ['SFLA', 'SFLU', 'SFLU', 'SFLU_1', 'RILD']
alias_dict_T['li_RLA'] = ['RLA3', 'RLA1', 'RLA5']

alias_dict_T['li_DEC_NP'] = ['NPHI', 'NPOR', 'NPHI_LS', 'NPOR_LS', 'NPHS', 'NPHI1', 'CNC'] # Neutron
alias_dict_T['li_DEC_DP'] = ['DPHZ2', 'DPHZ_LS', 'DPHI_LS', 'DPHI', 'DPOR', 'DPHI_SLDT', 'DPHZ', 'PORZ', 'DPO_LS', 'DPO'] # Density
alias_dict_T['li_DEC_ST'] = ['SPHI_SS', 'SPHI', 'TNPH_LS', 'SPHI_LS'] # Sonic

alias_dict_T['li_USF_S'] = ['DT1', 'DTSM_SLOW', 'DTSM_FAST', 'DT4S', 'DTRS', 'DTC', 'DTTS', 'DTS', 'DTMD', 'DTST', 'DT', 'DTOT', 'DTM'] # Shear
alias_dict_T['li_USF_P'] = ['DT4P', 'DTRP', 'DTTP', 'DTC', 'DTCO_1', 'DTCO', 'DT'] # Compressional

alias_dict_T['li_GAPI_D'] = ['GRD', 'SGRD', 'ECGRD', 'SGR'] # from Density
alias_dict_T['li_GAPI_R']  = ['ECGRR', 'GRR'] # from resistivity
alias_dict_T['li_GAPI_S'] = ['GRN', 'ECGRS'] # from sonic
alias_dict_T['li_GAPI_N'] = ['GR_EDTC', 'GR_STGC', 'GR']  # normal Gamma API

alias_dict_T['li_IN_D'] = ['C1', 'LCAL', 'CALI', 'CALD', 'LCALD'] # from density
alias_dict_T['li_IN_R'] = ['CALI_SPCS', 'CALR'] # from resistivity
alias_dict_T['li_IN_HILT'] =  ['HCAL2R', 'HCAL_1', 'HCAL_2', 'HCALD', 'HCALR', 'HCALS', 'HCAL'] # from HILT
 
alias_dict_T['li_LB_D'] = ['TEND'] # from density
alias_dict_T['li_LB_R'] = ['TENR'] # from Resistivity
alias_dict_T['li_LB_S'] = ['TENS'] # from Sonic

alias_dict_T['li_BE'] = ['PEF_SLDT', 'PE', 'PEFZ', 'PEF', 'PEFS', 'PEFL']

alias_dict_T['li_MV'] = ['SPR']


alias_T = {"DTSM" :['DTSM'],
           
         "DEC_NP"  : alias_dict_T['li_DEC_NP'],
         "DEC_DP"  : alias_dict_T['li_DEC_DP'],
         "DEC_ST"  : alias_dict_T['li_DEC_ST'],

         "USF_S"  : alias_dict_T['li_USF_S'],
         "USF_P"  : alias_dict_T['li_USF_P'],

         "GAPI_D" : alias_dict_T['li_GAPI_D'],
         "GAPI_R" : alias_dict_T['li_GAPI_R'],
         "GAPI_S" : alias_dict_T['li_GAPI_S'],
         "GAPI_N" : alias_dict_T['li_GAPI_N'],

         "IN_D"   : alias_dict_T['li_IN_D'],
         "IN_R"   : alias_dict_T['li_IN_R'],
         "IN_HILT"  : alias_dict_T['li_IN_HILT'],

         "MV"   : alias_dict_T['li_MV'],

         "BE"   : alias_dict_T['li_BE'],

          "SH"  : alias_dict_T['li_SH'],
          "DP"  : alias_dict_T['li_DP'],
         "SFL"  : alias_dict_T['li_SFL'],

         "LB_D"  : alias_dict_T['li_LB_D'],
         "LB_R"  : alias_dict_T['li_LB_R'],
         "LB_S"  : alias_dict_T['li_LB_S']
 }

alias_dict = np.array( [alias_T, alias_F] )

##### Training Data Download

In [1]:
#project welly training data download

import pickle

def load_object(obj):
  with open(obj, 'rb') as input:
      obj = pickle.load(input)
      return obj

if not os.path.exists('TrainingDataFT'):
  print("Downloading Data...")
  # use gitdown to upload train data from drive

data = np.array([
                  load_object('TrainingDataFT/TrainingdataForTrue.pkl'), 
                   load_object('TrainingDataFT/TrainingdataForFalse.pkl')
                ])


NameError: ignored

##### test data download

In [ ]:
#use gdown to upload test data

Downloading...
From: https://drive.google.com/uc?id=1-43M2dHo-p7sBMQv0DK2RK8ZC1cNpVUC
To: /content/final-data.zip
6.83MB [00:00, 60.2MB/s]


FileNotFoundError: ignored

# Main Class

In [ ]:
class WellLog:

  def __init__(self, filepath, alias_dict, data, valid = False, drop_col = [], thresh = 1 ):

    self.filepath = filepath
    self.test = []
    self.region = 'U'
    self.alias_dict = alias_dict
    self.data = data
    self.valid = valid
    self.drop_col = drop_col
    self.preds = []
    self.deploy = False
    self.ydep = []

    self.thresh = thresh

    self.cluster_region()
    self.test_util()

  def cluster_region( self):

    las = lasio.read(self.filepath)
    lat = las.well['SLAT'].value
    lon = las.well['SLON'].value
    cluster_centers = [[5.80127095, 3.52103194],
                          [2.27100945, 7.48227453]]
    dist = []
    for x in cluster_centers:
     dist.append( (lat-x[0])**2 +  (lon-x[1])**2 )
    if dist[0]<dist[1]:
      self.region = 'T'
    else:
      self.region = 'F'
    print('Region : {}'.format(self.region))

  def id(self,x):
    if x == 'T':
      return 0
    else:
      return 1

  def test_util( self):

    p = Project.from_las(self.filepath)
    self.test = p.df(keys = list(alias_dict[self.id(self.region)].keys()),
                     alias= self.alias_dict[self.id(self.region)] )
    self.test.dropna(axis=1, how='all', inplace=True)

    if 'DTSM' in self.test.columns:
      print("\nDEPLOY FILE\n")
      self.test = self.test[self.test['DTSM'].notna()]
      self.deploy = True
      self.ydep = self.test['DTSM']
      self.test.drop(columns= ['DTSM'], inplace=True)

    print(self.test.info())
    if len(self.drop_col)>0:
      print("CALLEDDDDDDDDDDDDDDDDDD")
      self.test_clean(self.drop_col,apply = 1)

  def test_clean(self, drop_col = [], apply = 0 ):
      
      temp = self.test.drop(columns = drop_col, axis =1)
      print(temp.info())
      if apply:
        self.test = temp

  def train(self, model = 'both'):

    xtrain = self.data[ self.id(self.region) ]
    col_list = list(self.test.columns)
    col_list.append('DTSM')
    xtrain = xtrain[col_list]
    col_list.pop()
    xtrain = xtrain[xtrain['DTSM'].notna()]

    thresh = int((len(col_list))//2)+ self.thresh
    xtrain = xtrain.dropna( thresh = thresh)
    ytrain = xtrain['DTSM']
    xtrain.drop(columns= ['DTSM'], inplace=True)
    
    print('Training data : {}'.format(len(xtrain)) )

    pred = []

    if model=='xgb' or model=='both':
      print("\nTraining XGB...")
      xgb_r = xg.XGBRegressor(objective='reg:squarederror',
                              random_state = 1,
                              tree_method = 'gpu_hist',
                            n_estimators =1000,
                            max_depth = 8)
      pred.append(self.train_util(xgb_r,xtrain,ytrain))

    if model=='cat' or model=='both':
      print("\nTraining CAT...")
      cat = CatBoostRegressor(verbose = 0,learning_rate  = 0.01,
                              task_type='GPU',
                              random_state = 1,
                              iterations = 1500, depth = 8)
      pred.append(self.train_util(cat, xtrain, ytrain))

    if (len(pred)>1):
      print("\nEnsembling...")

    if self.deploy:
      print('Deploy Weighted(1:3) RMSE :{}'.format( np.sqrt(MSE(self.ydep.values, (pred[0]+4*pred[1])/5)) ))
      print( (pred[0]+4*pred[1])/5 ) 

    if len(pred) >1:
      pred[0] = (pred[0]+4*pred[1])/5

    self.preds = pred[0]

  def train_util(self,model,xtrain,ytrain):

    if self.valid:
      xtrain, test_x, ytrain, test_y = train_test_split(xtrain, ytrain, 
                                                        test_size=0.2,
                                                        random_state= 42)
    a = time.time()
    model.fit( xtrain , ytrain )
    b = time.time()
    print('Train time :{}'.format(b-a))

    if self.valid:
      a = time.time()
      pred = model.predict(test_x)
      b = time.time()
      print('Valid time :{}'.format(b-a))
      rmse = np.sqrt(MSE(test_y, pred))
      print('Valid RMSE :{}'.format(rmse))

    pred = model.predict(self.test)

    if self.deploy:
      rmse = np.sqrt(MSE(self.ydep.values, pred))
      print('Deploy RMSE :{}'.format(rmse))
      print(pred)

    return pred

  def predictions(self, folder_dir= '', filename = ''):
    if len(filename) > 0:
      x = lasio.read(folder_dir + '/' +filename + '.las').df()
      x = x.index
      x = pd.DataFrame(x)
      x['DTSM'] = self.preds
      x.columns = ['Depth', 'DTSM']
      print(x.head())
      print(x.tail())
      x.to_excel(filename + '.xlsx', index=False)

    return self.preds

# Test a Single File

In [ ]:
# looping 

# x = [  

# '63250f7d463b_TGS.las',
# '638f2cc65681_TGS.las',
# 'fca03aa6acde_TGS.las',
# '3369b6f8fb6f_TGS.las',
# '8e37531ba266_TGS.las',
# 'eed1e9537976_TGS.las',
# '113412eec2a6_TGS.las',
# 'ff7845ea074d_TGS.las',
# '34a80ab7a5fa_TGS.las',
# '84c5fb9cc880_TGS.las',   // errors
# '7595ba9fb314_TGS.las',
# '302460e3021a_TGS.las', // errors
# '2f96a5f92418_TGS.las',
# '1684cc35f399_TGS.las',

# ]

x = [  

'7595ba9fb314_TGS.las',
'2f96a5f92418_TGS.las',
'1684cc35f399_TGS.las'

]


for filename in x:
    print(filename)
    test = WellLog( 'test-data/' + filename, alias_dict, data, valid=0  )
    test.train()
    test.predictions( 'test-data' ,filename[:-4] )

0it [00:00, ?it/s]

7595ba9fb314_TGS.las
Region : T


1it [00:00,  8.92it/s]


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1893 entries, ('', 3169.3104000000003) to ('', 3457.651199999893)
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DEC_NP  1893 non-null   float64
 1   USF_P   1893 non-null   float64
 2   GAPI_D  1893 non-null   float64
 3   GAPI_R  1893 non-null   float64
 4   IN_D    1893 non-null   float64
 5   MV      1893 non-null   float64
 6   BE      1893 non-null   float64
 7   DP      1893 non-null   float64
 8   SFL     1893 non-null   float64
 9   LB_D    1893 non-null   float64
 10  LB_R    1893 non-null   float64
 11  LB_S    1893 non-null   float64
dtypes: float64(12)
memory usage: 278.0+ KB
None
Training data : 329394

Training XGB...
Train time :15.142849683761597

Training CAT...
Train time :9.155544757843018

Ensembling...
     Depth        DTSM
0  10398.0  101.540324
1  10398.5  101.169375
2  10399.0  101.354125
3  10399.5  107.200554
4  10400.0  108.410917
        Depth

0it [00:00, ?it/s]

...

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3497 entries, ('', 3897.63) to ('', 4430.420399999802)
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DEC_NP   3478 non-null   float64
 1   DEC_DP   3497 non-null   float64
 2   USF_P    3495 non-null   float64
 3   GAPI_R   3463 non-null   float64
 4   IN_HILT  3488 non-null   float64
 5   MV       3497 non-null   float64
 6   BE       3489 non-null   float64
 7   DP       3485 non-null   float64
dtypes: float64(8)
memory usage: 416.3+ KB
None
Training data : 520947

Training XGB...
Train time :17.0696918964386

Training CAT...
Train time :10.662410974502563

Ensembling...
     Depth        DTSM
0  12787.5  102.356538
1  12788.0  103.625535
2  12788.5  102.120505
3  12789.0  101.664168
4  12789.5  102.434800
        Depth        DTSM
3492  14533.5  124.156153
3493  14534.0  126.466338
3494  14534.5  126.801896
3495  14535.0  127.696508
3496  14535.5  128.288589
1684cc35f

0it [00:00, ?it/s]

Region : T


1it [00:00,  1.74it/s]


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 13255 entries, ('', 1826.9712000000002) to ('', 3846.8807999992478)
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DEC_NP  9664 non-null   float64
 1   DEC_DP  11021 non-null  float64
 2   USF_P   13255 non-null  float64
 3   GAPI_R  13255 non-null  float64
 4   IN_D    11291 non-null  float64
 5   IN_R    10593 non-null  float64
 6   MV      13255 non-null  float64
 7   BE      7979 non-null   float64
 8   DP      13255 non-null  float64
 9   SFL     13255 non-null  float64
dtypes: float64(10)
memory usage: 1.8+ MB
None
Training data : 457864

Training XGB...
Train time :15.443778276443481

Training CAT...
Train time :10.156720876693726

Ensembling...
    Depth        DTSM
0  5994.0  133.372528
1  5994.5  131.308559
2  5995.0  130.000267
3  5995.5  121.773187
4  5996.0  116.169902
         Depth        DTSM
13250  12619.0  107.659834
13251  12619.5  107.537886
13252  1262

In [ ]:
# 00d02be79f49_TGS.las  USF
# ae16a9f64878_TGS.las  DP SFL 
# 94c1f5cae85c_TGS.las  BE
# ed48bda2217f_TGS.las  MD
# 20372701d5e2_TGS.las  DEC IN BE DP
# 0a7822c59487_TGS.las  IN BE SH MD

temp = dict()
temp['00d02be79f49_TGS.las'] = []
temp['ae16a9f64878_TGS.las'] = ['IN_D', 'DP', 'SFL']
temp['94c1f5cae85c_TGS.las'] = ['BE']
temp['ed48bda2217f_TGS.las'] = []
temp['20372701d5e2_TGS.las'] = ['DEC_DP', 'IN_HILT', 'BE', 'DP']
temp['0a7822c59487_TGS.las'] = ['DEC_T', 'IN_HILT', 'BE', 'SH', 'DP']


x = [
# '00d02be79f49_TGS.las',
# 'ae16a9f64878_TGS.las', error
# '94c1f5cae85c_TGS.las', error
# 'ed48bda2217f_TGS.las', error
'20372701d5e2_TGS.las',
'0a7822c59487_TGS.las'
]

for filename in x:
    print(filename)
    test = WellLog( 'test-data/' + filename, alias_dict, data, valid=0,
                   drop_col = temp[filename] )
    test.train()
    test.predictions( 'test-data' ,filename[:-4] )

0it [00:00, ?it/s]

20372701d5e2_TGS.las
Region : T


1it [00:00,  7.35it/s]


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2147 entries, ('', 3994.404) to ('', 4321.454399999879)
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DEC_NP   2101 non-null   float64
 1   DEC_DP   1 non-null      float64
 2   USF_P    2147 non-null   float64
 3   GAPI_R   2033 non-null   float64
 4   IN_HILT  1 non-null      float64
 5   MV       2147 non-null   float64
 6   BE       1 non-null      float64
 7   DP       1 non-null      float64
dtypes: float64(8)
memory usage: 237.4+ KB
None
CALLEDDDDDDDDDDDDDDDDDD
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2147 entries, ('', 3994.404) to ('', 4321.454399999879)
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DEC_NP  2101 non-null   float64
 1   USF_P   2147 non-null   float64
 2   GAPI_R  2033 non-null   float64
 3   MV      2147 non-null   float64
dtypes: float64(4)
memory usage: 170.3+ KB
None
Tr

0it [00:00, ?it/s]

Region : F


1it [00:00,  2.17it/s]


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5135 entries, ('', 1788.5664000000002) to ('', 2570.987999999709)
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DEC_D    4117 non-null   float64
 1   DEC_N    4199 non-null   float64
 2   DEC_T    185 non-null    float64
 3   DEC_S    5111 non-null   float64
 4   USF_P    5111 non-null   float64
 5   GAPI_D   4253 non-null   float64
 6   IN_R     4197 non-null   float64
 7   IN_HILT  211 non-null    float64
 8   BE       211 non-null    float64
 9   LBF_D    4461 non-null   float64
 10  LBF_S    5135 non-null   float64
 11  SH       231 non-null    float64
 12  DP       4197 non-null   float64
dtypes: float64(13)
memory usage: 736.9+ KB
None
CALLEDDDDDDDDDDDDDDDDDD
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5135 entries, ('', 1788.5664000000002) to ('', 2570.987999999709)
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  ----------

In [ ]:
# (self, filepath, alias_dict, data, valid = False, drop_col = [] )
test = WellLog( 'test-data/0a7822c59487_TGS.las', alias_dict, data, valid=0  )

0it [00:00, ?it/s]

Region : F


1it [00:00,  2.16it/s]

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5135 entries, ('', 1788.5664000000002) to ('', 2570.987999999709)
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DEC_D    4117 non-null   float64
 1   DEC_N    4199 non-null   float64
 2   DEC_T    185 non-null    float64
 3   DEC_S    5111 non-null   float64
 4   USF_P    5111 non-null   float64
 5   GAPI_D   4253 non-null   float64
 6   IN_R     4197 non-null   float64
 7   IN_HILT  211 non-null    float64
 8   BE       211 non-null    float64
 9   LBF_D    4461 non-null   float64
 10  LBF_S    5135 non-null   float64
 11  SH       231 non-null    float64
 12  DP       4197 non-null   float64
dtypes: float64(13)
memory usage: 736.9+ KB
None


In [ ]:
#  drop_col = [], apply = 0
# apply = 0, for temproray, 1 for permanenet
test.test_clean([], apply = 1)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1297 entries, ('', 4877.4096) to ('', 5074.919999999926)
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DEC_NP  1297 non-null   float64
 1   USF_P   1289 non-null   float64
 2   GAPI_R  1297 non-null   float64
 3   IN_D    1297 non-null   float64
 4   MV      1297 non-null   float64
dtypes: float64(5)
memory usage: 104.8+ KB
None


In [ ]:
#  model = 'both'
test.train()

Training data : 646070

Training XGB...
Train time :15.220812559127808

Training CAT...
Train time :11.674485921859741

Ensembling...


In [ ]:
# path + filename
test.predictions( 'test-data' ,'84c5fb9cc880_TGS')

ValueError: ignored

# Train Folder

In [ ]:
test_dir = 'wellF'
output_dir = 'outputwellf'

In [ ]:
drop_col_dict = dict()

In [ ]:
files = os.listdir(test_dir)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for filename in files:

  drop_col = []
  if filename in list( drop_col_dict.keys() ):
    drop_col = drop_col_dict[filename]

  test = WellLog( test_dir + '/' + filename , alias_dict, data, valid=1, drop_col  )
  test.train()
  test.predictions()

SyntaxError: ignored

#sfd

In [ ]:
x = [  

'63250f7d463b_TGS.las',
'638f2cc65681_TGS.las',
'fca03aa6acde_TGS.las',
'3369b6f8fb6f_TGS.las',
'8e37531ba266_TGS.las',
'eed1e9537976_TGS.las',
'113412eec2a6_TGS.las',
'ff7845ea074d_TGS.las',
'34a80ab7a5fa_TGS.las',
'7595ba9fb314_TGS.las',
'2f96a5f92418_TGS.las',
'1684cc35f399_TGS.las',

]

In [ ]:
!mkdir predict

mkdir: cannot create directory ‘predict’: File exists


In [ ]:
import shutil

for a in x:
  shutil.copy( a[:-4] + '.xlsx' , 'predict/' + a[:-4] + '.xlsx'  )

In [ ]:
shutil.make_archive( 'predict', 'zip', '/content/predict')

'/content/predict.zip'

In [ ]:
x = lasio.read('test-data/' +'84c5fb9cc880_TGS.las').df()

In [ ]:
x

,NPHI_LS,DTCO,PEFZ,DPHZ_LS,HCALD,HDRA,RHOZ,DRH,LCALD,DPO_LS,RHOM,GRR,ILD,ILM,SFLU,SPR,IDPH,IMPH
DEPT,,,,,,,,,,,,,,,,,,
16002.0,0.2363,99.2047,NaN,NaN,NaN,NaN,NaN,-0.0098,6.1436,0.0372,2.6473,91.1068,NaN,NaN,NaN,-36.1476,157.7764,2014.8407
16002.5,0.2400,96.7596,NaN,NaN,NaN,NaN,NaN,-0.0071,6.1390,0.0350,2.6496,92.4773,NaN,NaN,NaN,-37.1441,1979.7109,2008.0165
16003.0,0.2357,94.3550,NaN,NaN,NaN,NaN,NaN,-0.0067,6.1405,0.0352,2.6494,90.9638,NaN,NaN,NaN,-35.3410,1.9868,258.0079
16003.5,0.2263,93.4000,NaN,NaN,NaN,NaN,NaN,-0.0071,6.1447,0.0377,2.6472,89.6819,NaN,NaN,NaN,-32.7398,0.3409,3.1843
16004.0,0.2279,93.0141,NaN,NaN,NaN,NaN,NaN,-0.0079,6.1431,0.0431,2.6369,89.0719,NaN,NaN,NaN,-33.3049,0.3836,1.0418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16648.0,0.2224,85.5078,NaN,NaN,NaN,NaN,NaN,-0.0013,6.2277,0.0229,2.6725,97.8318,NaN,NaN,NaN,-96.4592,2.8262,3.3699
16648.5,0.2221,84.4351,NaN,NaN,NaN,NaN,NaN,-0.0016,6.2274,0.0230,2.6740,98.4990,NaN,NaN,NaN,-95.9137,2.8439,3.4092
16649.0,0.2216,83.4179,NaN,NaN,NaN,NaN,NaN,-0.0032,6.2272,0.0256,2.6619,99.6007,NaN,NaN,NaN,-96.2981,2.8721,3.2621


In [ ]:
p = Project.from_las('test-data/' +'84c5fb9cc880_TGS.las')
y = p.df(keys = list(alias_dict[0].keys()),
                    alias= alias_dict[0] )
y.dropna(axis = 0, how = 'all')
y['DEC_NP']

1it [00:00, 12.40it/s]


UWI  Depth    
     4877.4096    0.2363
     4877.5620    0.2400
     4877.7144    0.2357
     4877.8668    0.2263
     4878.0192    0.2279
                   ...  
     5074.3104    0.2224
     5074.4628    0.2221
     5074.6152    0.2216
     5074.7676    0.2218
     5074.9200    0.2191
Name: DEC_NP, Length: 1297, dtype: float64

In [ ]:
x = lasio.read('test-data/' + '84c5fb9cc880_TGS.las' ).df()
x.dropna(axis = 0, how = 'all')

,NPHI_LS,DTCO,PEFZ,DPHZ_LS,HCALD,HDRA,RHOZ,DRH,LCALD,DPO_LS,RHOM,GRR,ILD,ILM,SFLU,SPR,IDPH,IMPH
DEPT,,,,,,,,,,,,,,,,,,
16002.0,0.2363,99.2047,NaN,NaN,NaN,NaN,NaN,-0.0098,6.1436,0.0372,2.6473,91.1068,NaN,NaN,NaN,-36.1476,157.7764,2014.8407
16002.5,0.2400,96.7596,NaN,NaN,NaN,NaN,NaN,-0.0071,6.1390,0.0350,2.6496,92.4773,NaN,NaN,NaN,-37.1441,1979.7109,2008.0165
16003.0,0.2357,94.3550,NaN,NaN,NaN,NaN,NaN,-0.0067,6.1405,0.0352,2.6494,90.9638,NaN,NaN,NaN,-35.3410,1.9868,258.0079
16003.5,0.2263,93.4000,NaN,NaN,NaN,NaN,NaN,-0.0071,6.1447,0.0377,2.6472,89.6819,NaN,NaN,NaN,-32.7398,0.3409,3.1843
16004.0,0.2279,93.0141,NaN,NaN,NaN,NaN,NaN,-0.0079,6.1431,0.0431,2.6369,89.0719,NaN,NaN,NaN,-33.3049,0.3836,1.0418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16648.0,0.2224,85.5078,NaN,NaN,NaN,NaN,NaN,-0.0013,6.2277,0.0229,2.6725,97.8318,NaN,NaN,NaN,-96.4592,2.8262,3.3699
16648.5,0.2221,84.4351,NaN,NaN,NaN,NaN,NaN,-0.0016,6.2274,0.0230,2.6740,98.4990,NaN,NaN,NaN,-95.9137,2.8439,3.4092
16649.0,0.2216,83.4179,NaN,NaN,NaN,NaN,NaN,-0.0032,6.2272,0.0256,2.6619,99.6007,NaN,NaN,NaN,-96.2981,2.8721,3.2621


In [ ]:
x['NPHI_LS'] == z['DEC_NP']

ValueError: ignored

In [ ]:
for i in range(len(x['NPHI_LS'])):
  if x['NPHI_LS'][i] != 1:
    print(i)

In [ ]:
a = x['NPHI_LS'].values

In [ ]:
b = z['DEC_NP'].values

In [ ]:
print(len(a))
print(len(b))

In [ ]:
print(a[3])
print(b[3])

In [ ]:
for i in range(40):
  print(a[i], b[i])

In [ ]:
len(a)

In [ ]:

import os
#use gdown to upload data here

Downloading...
From: https://drive.google.com/uc?id=1NjpECBJcVdSxhY8EQ83KgCb-f2WheX5V
To: /content/wellF.zip
155MB [00:01, 147MB/s]


In [ ]:
# 00d02be79f49_TGS.las  USF
# ae16a9f64878_TGS.las  DP SFL 
# 94c1f5cae85c_TGS.las  BE
# ed48bda2217f_TGS.las  MD
# 20372701d5e2_TGS.las  DEC IN BE DP
# 0a7822c59487_TGS.las  IN BE SH MD 
